In [1]:
import numpy as np
import matplotlib.pyplot as plt
import SiPMWaveGen as swg
import scipy.signal as scisig
import CFDHitFinder as cfd

def CheckInCoincidenceWindow(hitStartUpstream,
                              hitListDownstream,
                              coincidenceWindowLowerLim,
                              coincidenceWindowUpperLim):
    coincidenceList = np.zeros(0)

    for i_ds in range(0, np.size(hitListDownstream)):
        timeDiff = hitListDownstream[i_ds] - hitStartUpstream;
        if timeDiff > coincidenceWindowLowerLim and timeDiff < coincidenceWindowUpperLim:
            coincidenceList = np.append(coincidenceList, hitListDownstream[i_ds])

    return coincidenceList


def TimeMatching(hitListUpstream,
                 hitListDownstream,
                 coincidenceWindowLowerLim,
                 coincidenceWindowUpperLim):
    matchedHitList = np.zeros([1,2])
    for i_us in range(0, np.size(hitListUpstream)):
        coincidenceList = CheckInCoincidenceWindow(hitStartUpstream=hitListUpstream[i_us],
                                                   hitListDownstream=hitListDownstream,
                                                   coincidenceWindowLowerLim=coincidenceWindowLowerLim,
                                                   coincidenceWindowUpperLim=coincidenceWindowUpperLim)
        for i_ds in range(0, np.size(coincidenceList)):
            matchedHitList = np.append(matchedHitList, [[(hitListUpstream[i_us]), (hitListDownstream[i_ds])]], 
                                       axis=0)
            hitListDownstream[i_ds] = 0;
            
    if np.shape(matchedHitList)[0] > 1:
        matchedHitList = np.delete(matchedHitList, (0), axis=0)
        
    return matchedHitList

In [36]:
timeSample = 0.2  # nanosec
nSamples = 200  # no. of sampling points in a trigger
speAmplitude = 0.15  # single photon-electron response amplitude in volt
noiseSigmaInVolt = speAmplitude / 20  # noise sigma
riseTime = 1.5  # nanosec
fallTime = 3  # nanosec
nBits = 12  # 12bit ADC
voltMin = -0.9  # minimum voltage ADC in volt
dynamicRange = 1  # dynamic range of ADC
offset = 2000  # offset ADC

cfdThreshold = 0.2  # CFD threshold
durationTheshold = 10  # hit finder alg param: duration of a detected peak to be considered a hit
adjDurationThreshold = 10  # hit finder alg param: duration between two detected peaks to be considered a single hit
nNoiseSigmaThreshold = 1.5  # adcThreshold = (noise threshold) * nNoiseSigmaThreshold + baseline
sgFilter = True  # using Golay-Savitzky filter 
sgWindow = 15  # sg sliding window
sgPolyOrder = 3  # sg smoothing polynomial order

for i in range(40):
    [t1, p1, true_p1] = swg.aDigitizedTrigger(dt=timeSample,
                                              nsamples=nSamples,
                                              speAmplitude=speAmplitude,
                                              noiseSigmaInVolt=noiseSigmaInVolt,
                                              riseTime=riseTime,
                                              fallTime=fallTime,
                                              nBits=nBits,
                                              voltMin=voltMin,
                                              dynamicRange=dynamicRange,
                                              offset=offset)

    [t2, p2, true_p2] = swg.aDigitizedTrigger(dt=timeSample,
                                              nsamples=nSamples,
                                              speAmplitude=speAmplitude,
                                              noiseSigmaInVolt=noiseSigmaInVolt,
                                              riseTime=riseTime,
                                              fallTime=fallTime,
                                              nBits=nBits,
                                              voltMin=voltMin,
                                              dynamicRange=dynamicRange,
                                              offset=offset)

    [hitList1, hitLogic1, baseline1, noiseSigma1] = cfd.HitFinder(p=p1,
                                                                  noiseSigmaInVolt=noiseSigmaInVolt,
                                                                  cfdThreshold=cfdThreshold,
                                                                  durationTheshold=durationTheshold,
                                                                  adjDurationThreshold=adjDurationThreshold,
                                                                  nNoiseSigmaThreshold=nNoiseSigmaThreshold,
                                                                  sgFilter=sgFilter,
                                                                  sgWindow=sgWindow,
                                                                  sgPolyOrder=sgPolyOrder)

    [hitList2, hitLogic2, baseline2, noiseSigma2] = cfd.HitFinder(p=p2,
                                                                  noiseSigmaInVolt=noiseSigmaInVolt,
                                                                  cfdThreshold=cfdThreshold,
                                                                  durationTheshold=durationTheshold,
                                                                  adjDurationThreshold=adjDurationThreshold,
                                                                  nNoiseSigmaThreshold=nNoiseSigmaThreshold,
                                                                  sgFilter=sgFilter,
                                                                  sgWindow=sgWindow,
                                                                  sgPolyOrder=sgPolyOrder)

    matchedHitList = TimeMatching(hitListUpstream=hitList1*timeSample,
                                  hitListDownstream=hitList2*timeSample,
                                  coincidenceWindowLowerLim=25,
                                  coincidenceWindowUpperLim=35)

    plt.figure(1)

    plt.subplot(211)
    plt.plot(t1, p1, 'r')
    plt.ylim(offset, offset + 2 ** nBits)
    for x in hitList1:
        plt.axvline(x=(x * timeSample))
    for j in range(0, np.shape(matchedHitList)[0]):
        if matchedHitList[j, 0] == 0 and matchedHitList[j, 1] == 0:
            continue
        x1 = matchedHitList[j, 0]
        x2 = matchedHitList[j, 1]
        plt.fill_betweenx(y=range(offset, offset + 2**nBits),
                          x1=x1, 
                          x2=x2, 
                          facecolor='green', 
                          alpha=0.5)

    plt.subplot(212)
    plt.plot(t2, p2, 'r')
    plt.ylim(offset, offset + 2 ** nBits)
    for x in hitList2:
        plt.axvline(x=(x * timeSample))
    for j in range(0, np.shape(matchedHitList)[0]):
        if matchedHitList[j, 0] == 0 and matchedHitList[j, 1] == 0:
            continue
        x1 = matchedHitList[j, 0]
        x2 = matchedHitList[j, 1]
        plt.fill_betweenx(y=range(offset, offset + 2**nBits),
                          x1=x1, 
                          x2=x2, 
                          facecolor='green', 
                          alpha=0.5)

    plt.show()
